In [1]:
import pandas as pd
import json

In [31]:
## sorted sort -k2 -k4 -r -n <paired_concepts_test_file.txt >> pc_sorted_test.txt
paired_concepts_table = pd.read_csv('pc_sorted_test.txt', sep="\t")
paired_concepts_table

,dataset_id,concept_id_1,concept_id_2,concept_count,concept_prevalence,chi_square_t,chi_square_p,expected_count,ln_ratio,rel_freq_1,rel_freq_2
0,1,23325,907206,10,0.000006,90.965850,1.461735e-21,0.912427,2.394232,0.001139,0.053763
1,1,23325,903222,10,0.000006,38.133000,6.608295e-10,1.780705,1.725576,0.001139,0.027548
2,1,23325,766928,10,0.000006,21.866740,2.922554e-06,2.550872,1.366150,0.001139,0.019231
3,1,23325,766851,10,0.000006,5.721939,1.675422e-02,4.782885,0.737541,0.001139,0.010256
4,1,23325,766821,10,0.000006,4.842118,2.777276e-02,5.062499,0.680725,0.001139,0.009690
...,...,...,...,...,...,...,...,...,...,...,...
9994,1,8507,974677,8,0.000004,2.014034,1.558505e-01,5.473665,0.379493,0.000011,0.615385
9995,1,8507,924939,8,0.000004,0.036359,8.487762e-01,8.421023,-0.051290,0.000011,0.400000
9996,1,8507,990029,7,0.000004,2.040442,1.531652e-01,10.526280,-0.407965,0.000009,0.280000
9997,1,8507,985786,7,0.000004,0.414192,5.198494e-01,8.421023,-0.184821,0.000009,0.350000


In [6]:
with open('concept_xref.json') as f:
  xref_data = json.load(f)

In [8]:
xref_data[0]

{'_id': '197339',
 'concept_name': 'Congenital abnormality of uterus, affecting pregnancy',
 'domain_id': 'Condition',
 'concept_class_id': 'Clinical Finding',
 'xrefs': {'SNOMED': '41215002',
  'SNOMEDCT': '41215002',
  'ICD9CM': ['654.0', '654.00'],
  'UMLS': ['C0269727', 'C0157006']}}

In [13]:
xref_data_dict = {}
for x in xref_data:
    xref_data_dict[x["_id"]] = x
xref_data_dict['197339']

{'_id': '197339',
 'concept_name': 'Congenital abnormality of uterus, affecting pregnancy',
 'domain_id': 'Condition',
 'concept_class_id': 'Clinical Finding',
 'xrefs': {'SNOMED': '41215002',
  'SNOMEDCT': '41215002',
  'ICD9CM': ['654.0', '654.00'],
  'UMLS': ['C0269727', 'C0157006']}}

## API 1

In [126]:

paired_concept_file_path = 'pc_sorted_test.txt'
paired_concept_column_names = ["dataset_id","concept_id_1","concept_id_2","concept_count","concept_prevalence","chi_square_t","chi_square_p","expected_count","ln_ratio","rel_freq_1","rel_freq_2"]

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

def generate_results(concept_dict):
    assoc_id = int(concept_dict["concept_id_2"])
    current_result = {
            "associated_concept_id": assoc_id,
            "associated_concept_name": xref_data_dict[str(assoc_id)]["concept_name"],
            "associated_domain_id": xref_data_dict[str(assoc_id)]['domain_id'],
            "concept_count": int(concept_dict["concept_count"]),
            "concept_prevalence": concept_dict["concept_prevalence"],
            "dataset_id": concept_dict["dataset_id"],
            "chi_square_t": concept_dict["chi_square_t"],
            "chi_square_p": concept_dict["chi_square_p"],
            "ln_ratio": concept_dict["ln_ratio"],
            "rel_freq_1": concept_dict["rel_freq_1"],
            "rel_freq_2":concept_dict["rel_freq_2"]
    }
    return current_result 

def load_annotations():
    chunk_size = 1000000
    max_combos = 100
    paired_concepts_table_total = pd.read_csv(paired_concept_file_path, sep='\t', header=None, names= paired_concept_column_names, chunksize=chunk_size)  
    first_chunk = True
    row_counter = 0
    row_total = file_len(paired_concept_file_path)
    extra_entry = False
    for chunk in paired_concepts_table_total:
        paired_concepts_table = chunk
        for i,j in paired_concepts_table.iterrows():
            row_counter = row_counter + 1
            current_id = int(j["concept_id_1"])
            if(first_chunk):
                last_id = int(paired_concepts_table.iloc[0]["concept_id_1"])
                current_results = []
                current_results.append(generate_results(paired_concepts_table.iloc[0]))
                current_count = 1
                first_chunk = False
            elif((current_count < max_combos) & (last_id == current_id) & (row_counter != (row_total - 1))):
                current_results.append(generate_results(j))
                current_count = current_count + 1
            elif((last_id != current_id) | (i == (row_total - 1))):
                last_id_results = current_results
                if((current_count < max_combos)&(last_id == current_id)&(i == (row_total - 1))):
                    last_id_results.append(generate_results(j))
                elif((last_id != current_id) & (i == (row_total - 1))):
                    extra_entry = True
                    extra_dict = {
                        "_id": current_id,
                        "concept_name": xref_data_dict[str(current_id)]["concept_name"],
                        "domain_id": xref_data_dict[str(current_id)]["domain_id"],
                        "xrefs": xref_data_dict[str(current_id)],
                        "results": [generate_results(j)]
                    }
                else:
                    current_results = []
                    current_results.append(generate_results(j))
                    current_count = 1
                current_dict = {
                    "_id": last_id,
                    "concept_name": xref_data_dict[str(last_id)]["concept_name"],
                    "domain_id": xref_data_dict[str(last_id)]["domain_id"],
                    "xrefs": xref_data_dict[str(last_id)],
                    "results": last_id_results
                }
#                 print(current_dict)
                yield(current_dict)
            last_id = current_id
        if(extra_entry):
#             print(extra_dict)
            yield(extra_dict)

load_annotations()

<generator object load_annotations at 0x7fe414129f90>

## API 2

In [127]:
def load_annotations():
    chunk_size = 1000
    paired_concepts_table_total = pd.read_csv(paired_concept_file_path, sep='\t', header=None, names= paired_concept_column_names, chunksize=chunk_size)  
    for chunk in paired_concepts_table_total:
        paired_concepts_table = chunk
        for i,j in paired_concepts_table.iterrows():
            current_id_1 = str(int(j["concept_id_1"]))
            current_id_2 = str(int(j["concept_id_2"]))
            current_dict = {
              "_id": current_id_1 + "-" + current_id_2,
              "concept1": {
                 "omop": current_id_1,
                 "xrefs": xref_data_dict[current_id_1]["xrefs"]
              },
              "concept2": {
                 "omop": current_id_2,
                 "xrefs": xref_data_dict[current_id_2]["xrefs"]
              },
              "results": [
                {
                    "concept_count": int(j["concept_count"]),
                    "concept_prevalence": j["concept_prevalence"],
                    "dataset_id": j["dataset_id"],
                    "chi_square_t": j["chi_square_t"],
                    "chi_square_p": j["chi_square_p"],
                    "ln_ratio": j["ln_ratio"],
                    "rel_freq_1": j["rel_freq_1"],
                    "rel_freq_2":j["rel_freq_2"]
                }
              ]
            }
#             print(current_dict)
            yield(current_dict)
            
load_annotations()

<generator object load_annotations at 0x7fe412e0cd60>